<a href="https://colab.research.google.com/github/matsudo840/nlp100/blob/master/Chap3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語処理100本ノック 2015

## 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

1行に1記事の情報がJSON形式で格納される
各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
ファイル全体はgzipで圧縮される
以下の処理を行うプログラムを作成せよ．

In [1]:
!wget -O /content/jawiki-country.json.gz http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz
!gunzip -c jawiki-country.json.gz > jawiki-country.json
!ls

--2019-04-18 06:26:32--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3605822 (3.4M) [application/x-gzip]
Saving to: ‘/content/jawiki-country.json.gz’

/content/jawiki-cou 100%[===================>]   3.44M  2.70MB/s    in 1.3s    

2019-04-18 06:26:34 (2.70 MB/s) - ‘/content/jawiki-country.json.gz’ saved [3605822/3605822]

jawiki-country.json  jawiki-country.json.gz  sample_data


### 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [0]:
import json

with open('jawiki-country.json', 'r') as f:
  for entry in f:
    entry_json = json.loads(entry)
    if entry_json['title'] == 'イギリス':
      entry_england = entry_json
      print(entry_england['text'])
      break

### 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [3]:
import re

m = '^\[\[Category:.*\]\]$'
for line in entry_england['text'].split('\n'):
  if re.search(m, line):
    print(line)

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


### 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [4]:
m2 = r'^\[\[Category:(.*?)[\|\]]'
for line2 in entry_england['text'].split('\n'):
  m2_match = re.search(m2, line2)
  if m2_match:
    print(m2_match.group(1))

イギリス
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国
1801年に設立された州・地域


### 23. セクション構造

記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [5]:
m3 = r'(=+)(.*?)=='
for line3 in entry_england['text'].split('\n'):
  m3_match = re.match(m3, line3)
  if m3_match:
    print(f'{m3_match.group(2)}: {len(m3_match.group(1))-1}')  

国名: 1
歴史: 1
地理: 1
気候: 2
政治: 1
外交と軍事: 1
地方行政区分: 1
主要都市: 2
科学技術: 1
経済: 1
鉱業: 2
農業: 2
貿易: 2
通貨: 2
企業: 2
交通: 1
道路: 2
鉄道: 2
海運: 2
航空: 2
通信: 1
国民: 1
言語: 2
宗教: 2
 婚姻 : 2
教育: 2
文化: 1
食文化: 2
文学: 2
 哲学 : 2
音楽: 2
イギリスのポピュラー音楽: 3
映画: 2
コメディ: 2
国花: 2
世界遺産: 2
祝祭日: 2
スポーツ: 1
サッカー: 2
競馬: 2
モータースポーツ: 2
脚注: 1
関連項目: 1
外部リンク: 1


### 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [6]:
list = re.findall(r'(?:file|ファイル):(.+?)\|', entry_england['text'])

list

['Royal Coat of Arms of the United Kingdom.svg',
 'CHANDOS3.jpg',
 'The Fabs.JPG',
 'PalaceOfWestminsterAtNight.jpg',
 'Westminster Abbey - West Door.jpg',
 'Edinburgh Cockburn St dsc06789.jpg',
 'Canterbury Cathedral - Portal Nave Cross-spire.jpeg',
 'Kew Gardens Palm House, London - July 2009.jpg',
 '2005-06-27 - United Kingdom - England - London - Greenwich.jpg',
 'Stonehenge2007 07 30.jpg',
 'Yard2.jpg',
 'Durham Kathedrale Nahaufnahme.jpg',
 'Roman Baths in Bath Spa, England - July 2006.jpg',
 'Fountains Abbey view02 2005-08-27.jpg',
 'Blenheim Palace IMG 3673.JPG',
 'Liverpool Pier Head by night.jpg',
 "Hadrian's Wall view near Greenhead.jpg",
 'London Tower (1).JPG',
 'Wembley Stadium, illuminated.jpg']

### 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [7]:
s = re.search(r'{{基礎情報.*?\n\|(.*)\n}}', entry_england['text'], re.DOTALL)

dict = {}
for elm in s.group(1).split('\n|'):
  lst = elm.split(' = ')
  dict[lst[0]] = lst[1]

dict

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': '[[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': '[[エリザベス2世]]',


### 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [8]:
s = re.search(r'{{基礎情報.*?\n\|(.*)\n}}', entry_england['text'], re.DOTALL)

dict = {}
for elm in s.group(1).split('\n|'):
  lst = elm.split(' = ')
  dict[lst[0]] = re.sub(r"\'{2,5}'", "", lst[1])

dict

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': '[[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': '[[エリザベス2世]]',


### 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [9]:
s = re.search(r'{{基礎情報.*?\n\|(.*)\n}}', entry_england['text'], re.DOTALL)

dict = {}
for elm in s.group(1).split('\n|'):
  lst = elm.split(' = ')
  dict[lst[0]] = re.sub(r"\'{2,5}'", "", lst[1])
  dict[lst[0]] = re.sub(r"\[\[(?:[^\|\[\]]*?\|)?([^\|\[\]]*?)\]\]", r'\1', dict[lst[0]])

dict

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': '.uk / .gb<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': 'エリザベス2世',
 '元首等肩書': '女

### 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [10]:
s = re.search(r'{{基礎情報.*?\n\|(.*)\n}}', entry_england['text'], re.DOTALL)

dict = {}
for elm in s.group(1).split('\n|'):
  lst = elm.split(' = ')
  dict[lst[0]] = re.sub(r"\'{2,5}'", "", lst[1])
  dict[lst[0]] = re.sub(r"\[\[(?:[^\|\[\]]*?\|)?([^\|\[\]]*?)\]\]", r'\1', dict[lst[0]])
  dict[lst[0]] = re.sub(r"<.*?>", "", dict[lst[0]])
  dict[lst[0]] = re.sub(r"\[\[.*?\]\]", "", dict[lst[0]])
  dict[lst[0]] = re.sub(r"\[.*?\]", "", dict[lst[0]])
  dict[lst[0]] = re.sub(r"{{(?:[^{}\|]*\|)+([^{}\|]*)}}", r'\1', dict[lst[0]])
  
dict

{'GDP/人': '36,727',
 'GDP値': '2兆3162億',
 'GDP値MER': '2兆4337億',
 'GDP値元': '1兆5478億',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': '.uk / .gb使用は.ukに比べ圧倒的少数。',
 '人口値': '63,181,775',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': 'エリザベス2世',
 '元首等肩書': '女王',
 '公式国名': 'United Kingdom of Great Britain and Northern Ireland英語以外での正式国名:\n*An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath（スコットランド・ゲール語）\n*Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon（ウェールズ語）\n*Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann（アイルランド語）\n*An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh（コーンウォール語）\n*Unitit Kinrick o Great Breetain an Northren Ireland（スコットランド語）\n**Claught Kängrick o Docht Brätain an Norlin Airlann、Unitet Kängdom o Great Brittain an Norlin Airlann（アルスター・スコットランド語）',
 '公用語': '英語（事実上）',
 '国旗画像': 'Flag of the United Kingd

### 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [36]:
import requests
import json

url = "https://www.mediawiki.org/w/api.php"

params = {
    "action":"query",
    "format":"json",
    "prop": "imageinfo",
    "titles":"File:Flag of the United Kingdom.svg",
    "iiprop":"url"
}

response = requests.get(url, params=params)
data = response.json()

print(data['query']['pages']['-1']['imageinfo'][0]['url'])


https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
